In [15]:
import pandas as pd

# Load bond data
bond_data = pd.read_csv(r'C:\Users\Melanie\Desktop\optimization_engine\bond_data.csv')

# Summary statistics for OAS (Option-Adjusted Spread)
print("OAS Statistics:")
print(bond_data['OAS'].describe())

# Calculate the sum of absolute deviations from the benchmark weight for MaxRisk guidance
BenchmarkWeight = 0.01
max_risk_estimate = (bond_data['OAS'] * abs(BenchmarkWeight)).sum()

print(f"Estimated MaxRisk based on benchmark weights: {max_risk_estimate}")

# Summary statistics for liquidity score
print("\nLiquidity Score Statistics:")
liq = (bond_data['liquidity_score'] * abs(BenchmarkWeight)).sum()
print(liq)

# Summary statistics for duration (used in volume and DTS constraints)
print("\nDuration Statistics:")
print(bond_data['duration'].describe())

# Summary statistics for DTS (Duration Times Spread)
bond_data['DTS'] = bond_data['duration'] * bond_data['OAS']
dts = (bond_data['DTS'] * abs(BenchmarkWeight)).sum()
print("\nDTS Statistics:")
print(dts)
print(bond_data['DTS'].describe())

# costs 
bond_data['transaction_cost'] = bond_data['ask_price'] - bond_data['bid_price']
t_cost = (bond_data['transaction_cost'] * abs(BenchmarkWeight)).sum()
print(t_cost)


OAS Statistics:
count    36500.000000
mean      1061.991507
std        928.239002
min          1.000000
25%        330.000000
50%        728.000000
75%       1582.000000
max       4575.000000
Name: OAS, dtype: float64
Estimated MaxRisk based on benchmark weights: 387626.89999999997

Liquidity Score Statistics:
2031.0660999999998

Duration Statistics:
count    36500.000000
mean         6.841742
std          3.244490
min          0.000000
25%          4.356493
50%          6.528530
75%          8.930026
max         17.711061
Name: duration, dtype: float64

DTS Statistics:
2273631.934057838
count    36500.000000
mean      6229.128586
std       4184.799268
min          0.000000
25%       2502.288161
50%       5959.507543
75%       9642.949192
max      24393.628774
Name: DTS, dtype: float64
334.013


In [29]:
import gurobipy as gp
from gurobipy import GRB
import pandas as pd

bond_data = pd.read_csv(r'C:\Users\Melanie\Desktop\optimization_engine\bond_data.csv')

BenchmarkWeight = 0.01 

bond_data['transaction_cost'] = bond_data['ask_price'] - bond_data['bid_price']
bond_data['daily_return'] = bond_data['expected_return']
bond_data['DTS'] = bond_data['duration'] * bond_data['OAS']

N = 100 
lambda_1 = 10
lambda_2 = 20

model = gp.Model("DynamicCorporateBondOptimization")

w = model.addVars(N, vtype=GRB.CONTINUOUS, lb=0, name="w")  
x = model.addVars(N, vtype=GRB.BINARY, name="x")  # whether bond i is chosen

model.setObjective(
    gp.quicksum((bond_data.loc[i, 'expected_return'] * w[i] - bond_data.loc[i, 'transaction_cost'] * x[i]) for i in range(N))  # return minus transaction cost
    - lambda_1 * gp.quicksum(bond_data.loc[i, 'OAS'] * w[i] for i in range(N))  # OAS risk term
    - lambda_2 * gp.quicksum(bond_data.loc[i, 'DTS'] * w[i] for i in range(N)),  # DTS risk term
    GRB.MAXIMIZE
)

# Risk constraints
benchmark_OAS = 274.88 
lower_bound = 0.9 * benchmark_OAS
upper_bound = 1.1 * benchmark_OAS
model.addConstr(gp.quicksum(bond_data.loc[i, 'OAS'] * w[i] for i in range(N)) >= lower_bound, "MinOAS")
model.addConstr(gp.quicksum(bond_data.loc[i, 'OAS'] * w[i] for i in range(N)) <= upper_bound, "MaxOAS")

# Liquidity
Liquidity = gp.quicksum(bond_data.loc[i, 'liquidity_score'] * BenchmarkWeight for i in range(N))
MinLiquidity = 0.9 * Liquidity
model.addConstr(gp.quicksum(bond_data.loc[i, 'liquidity_score'] * w[i] for i in range(N)) >= MinLiquidity, "MinLiquidity")

# Transaction cost 
Benchmark_cost = gp.quicksum(bond_data.loc[i, 'transaction_cost'] * BenchmarkWeight for i in range(N))
lower_t_cost = 0.9 * Benchmark_cost
upper_t_cost = 1.1 * Benchmark_cost
model.addConstr(gp.quicksum(bond_data.loc[i, 'transaction_cost'] * x[i] for i in range(N)) >= lower_t_cost, "MintCost")
model.addConstr(gp.quicksum(bond_data.loc[i, 'transaction_cost'] * x[i] for i in range(N)) <= upper_t_cost, "MaxtCost")

# binary 
M = 100000
for i in range(N):
    model.addConstr(w[i] <= M * x[i], f"WeightSelection_{i}")

# sum of weights = 1
model.addConstr(gp.quicksum(w[i] for i in range(N)) == 1, "WeightSum")

model.optimize()

if model.status == GRB.OPTIMAL:
    print("Optimal solution found:")
    for i in range(N):
        print(f"Bond {bond_data.loc[i, 'ISIN']}: weight = {w[i].x:.4f}, chosen = {x[i].x:.0f}")
else:
    print("No optimal solution found.")

chosen_bonds = [bond_data.loc[i, 'ISIN'] for i in range(N) if x[i].x == 1]
if chosen_bonds:
    print("Chosen bonds:", chosen_bonds)
    #output weights of the chosen_bonds
    print("Weights of chosen bonds:", [w[i].x for i in range(N) if x[i].x == 1])
else:
    print("No bonds were chosen.")
    
#

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 11+.0 (22631.2))

CPU model: Intel(R) Core(TM) i7-10750H CPU @ 2.60GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 106 rows, 200 columns and 800 nonzeros
Model fingerprint: 0xbfdb25a0
Variable types: 100 continuous, 100 integer (100 binary)
Coefficient statistics:
  Matrix range     [5e-02, 1e+05]
  Objective range  [3e-01, 1e+05]
  Bounds range     [1e+00, 1e+00]
  RHS range        [9e-01, 3e+02]
Presolve removed 66 rows and 128 columns
Presolve time: 0.00s
Presolved: 40 rows, 72 columns, 215 nonzeros
Variable types: 36 continuous, 36 integer (36 binary)
Found heuristic solution: objective -62824.98035
Found heuristic solution: objective -62499.87619
Found heuristic solution: objective -22687.32503

Root relaxation: objective -1.485186e+04, 12 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objec